# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 17 2023 3:01PM,261236,19,COR00003,"Coronado Aesthetics, LLC",Released
1,Apr 17 2023 2:59PM,261235,19,MSP220807,"Methapharm, Inc.",Released
2,Apr 17 2023 2:41PM,260872,20,REL1-5771,Reliable 1 Laboratories LLC,Released
3,Apr 17 2023 2:41PM,260872,20,REL1-5772,Reliable 1 Laboratories LLC,Released
4,Apr 17 2023 2:41PM,260872,20,REL1-5773,Reliable 1 Laboratories LLC,Released
5,Apr 17 2023 2:41PM,260872,20,REL1-5774,Reliable 1 Laboratories LLC,Released
6,Apr 17 2023 2:41PM,260872,20,REL1-5775,Reliable 1 Laboratories LLC,Released
7,Apr 17 2023 2:41PM,260872,20,REL1-5776,Reliable 1 Laboratories LLC,Released
8,Apr 17 2023 2:41PM,260872,20,REL1-5777,Reliable 1 Laboratories LLC,Released
9,Apr 17 2023 2:41PM,260872,20,REL1-5778,Reliable 1 Laboratories LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,261232,Released,1
26,261233,Released,1
27,261234,Released,1
28,261235,Released,1
29,261236,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261232,NaN,NaN,1.0
261233,NaN,NaN,1.0
261234,NaN,NaN,1.0
261235,NaN,NaN,1.0
261236,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260872,0.0,0.0,53.0
261146,0.0,1.0,0.0
261161,0.0,0.0,1.0
261179,0.0,0.0,1.0
261183,1.0,26.0,19.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260872,0,0,53
261146,0,1,0
261161,0,0,1
261179,0,0,1
261183,1,26,19


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260872,0,0,53
1,261146,0,1,0
2,261161,0,0,1
3,261179,0,0,1
4,261183,1,26,19


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260872,,,53
1,261146,,1,
2,261161,,,1
3,261179,,,1
4,261183,1,26,19


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 17 2023 3:01PM,261236,19,"Coronado Aesthetics, LLC"
1,Apr 17 2023 2:59PM,261235,19,"Methapharm, Inc."
2,Apr 17 2023 2:41PM,260872,20,Reliable 1 Laboratories LLC
55,Apr 17 2023 2:22PM,261234,19,"AdvaGen Pharma, Ltd"
56,Apr 17 2023 1:56PM,261233,10,"Virtue Rx, LLC"
57,Apr 17 2023 1:55PM,261232,19,"AdvaGen Pharma, Ltd"
58,Apr 17 2023 1:52PM,261216,15,"Brookfield Pharmaceuticals, LLC"
87,Apr 17 2023 1:44PM,261231,19,"AdvaGen Pharma, Ltd"
99,Apr 17 2023 1:36PM,261230,10,"Uniderm USA, Inc"
100,Apr 17 2023 1:19PM,261226,20,"Exact-Rx, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 17 2023 3:01PM,261236,19,"Coronado Aesthetics, LLC",,,1
1,Apr 17 2023 2:59PM,261235,19,"Methapharm, Inc.",,,1
2,Apr 17 2023 2:41PM,260872,20,Reliable 1 Laboratories LLC,,,53
3,Apr 17 2023 2:22PM,261234,19,"AdvaGen Pharma, Ltd",,,1
4,Apr 17 2023 1:56PM,261233,10,"Virtue Rx, LLC",,,1
5,Apr 17 2023 1:55PM,261232,19,"AdvaGen Pharma, Ltd",,,1
6,Apr 17 2023 1:52PM,261216,15,"Brookfield Pharmaceuticals, LLC",,,29
7,Apr 17 2023 1:44PM,261231,19,"AdvaGen Pharma, Ltd",,8,4
8,Apr 17 2023 1:36PM,261230,10,"Uniderm USA, Inc",,,1
9,Apr 17 2023 1:19PM,261226,20,"Exact-Rx, Inc.",,20,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 17 2023 3:01PM,261236,19,"Coronado Aesthetics, LLC",1,,
1,Apr 17 2023 2:59PM,261235,19,"Methapharm, Inc.",1,,
2,Apr 17 2023 2:41PM,260872,20,Reliable 1 Laboratories LLC,53,,
3,Apr 17 2023 2:22PM,261234,19,"AdvaGen Pharma, Ltd",1,,
4,Apr 17 2023 1:56PM,261233,10,"Virtue Rx, LLC",1,,
5,Apr 17 2023 1:55PM,261232,19,"AdvaGen Pharma, Ltd",1,,
6,Apr 17 2023 1:52PM,261216,15,"Brookfield Pharmaceuticals, LLC",29,,
7,Apr 17 2023 1:44PM,261231,19,"AdvaGen Pharma, Ltd",4,8,
8,Apr 17 2023 1:36PM,261230,10,"Uniderm USA, Inc",1,,
9,Apr 17 2023 1:19PM,261226,20,"Exact-Rx, Inc.",1,20,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 17 2023 3:01PM,261236,19,"Coronado Aesthetics, LLC",1,,
1,Apr 17 2023 2:59PM,261235,19,"Methapharm, Inc.",1,,
2,Apr 17 2023 2:41PM,260872,20,Reliable 1 Laboratories LLC,53,,
3,Apr 17 2023 2:22PM,261234,19,"AdvaGen Pharma, Ltd",1,,
4,Apr 17 2023 1:56PM,261233,10,"Virtue Rx, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 17 2023 3:01PM,261236,19,"Coronado Aesthetics, LLC",1.0,NaN,NaN
1,Apr 17 2023 2:59PM,261235,19,"Methapharm, Inc.",1.0,NaN,NaN
2,Apr 17 2023 2:41PM,260872,20,Reliable 1 Laboratories LLC,53.0,NaN,NaN
3,Apr 17 2023 2:22PM,261234,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
4,Apr 17 2023 1:56PM,261233,10,"Virtue Rx, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 17 2023 3:01PM,261236,19,"Coronado Aesthetics, LLC",1.0,0.0,0.0
1,Apr 17 2023 2:59PM,261235,19,"Methapharm, Inc.",1.0,0.0,0.0
2,Apr 17 2023 2:41PM,260872,20,Reliable 1 Laboratories LLC,53.0,0.0,0.0
3,Apr 17 2023 2:22PM,261234,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
4,Apr 17 2023 1:56PM,261233,10,"Virtue Rx, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2873374,40.0,27.0,1.0
12,261209,1.0,0.0,0.0
15,261216,29.0,0.0,0.0
16,261194,0.0,1.0,0.0
18,261161,1.0,0.0,0.0
19,2350893,11.0,9.0,0.0
20,522098,54.0,20.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2873374,40.0,27.0,1.0
1,12,261209,1.0,0.0,0.0
2,15,261216,29.0,0.0,0.0
3,16,261194,0.0,1.0,0.0
4,18,261161,1.0,0.0,0.0
5,19,2350893,11.0,9.0,0.0
6,20,522098,54.0,20.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,40.0,27.0,1.0
1,12,1.0,0.0,0.0
2,15,29.0,0.0,0.0
3,16,0.0,1.0,0.0
4,18,1.0,0.0,0.0
5,19,11.0,9.0,0.0
6,20,54.0,20.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,40.0
1,12,Released,1.0
2,15,Released,29.0
3,16,Released,0.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20
Status,,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,27.0,0.0,0.0,1.0,0.0,9.0,20.0
Released,40.0,1.0,29.0,0.0,1.0,11.0,54.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,27.0,0.0,0.0,1.0,0.0,9.0,20.0
2,Released,40.0,1.0,29.0,0.0,1.0,11.0,54.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,27.0,0.0,0.0,1.0,0.0,9.0,20.0
2,Released,40.0,1.0,29.0,0.0,1.0,11.0,54.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()